In [39]:
# Data exploration
import pandas as pd

# Numerical
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
#import required Python scripts to access their functions
import NYC_GetCleaned_HistoricData 
import data_utility
import NYC_GetCleaned_TotalPopulation

In [3]:
#import the functions from their corresponding files
from NYC_GetCleaned_HistoricData import getCleanedData
from NYC_GetCleaned_TotalPopulation import getMeanPopulation

In [4]:
#Get cleaned data from NYC_GetCleaned_HistoricData
crimes_original = getCleanedData()
crimes_original.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,3/30/2014,20:40:00,3/30/2014,20:50:00,3/30/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,INTENT TO",ATTEMPTED,FELONY,BRONX,44,INSIDE,RESIDENCE - APT. HOUSE,40.8362,-73.9152
1,4/16/2014,22:10:00,4/16/2014,22:12:00,4/17/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE,POSSESS.",ATTEMPTED,FELONY,BRONX,47,INSIDE,STREET,40.8947,-73.8614
2,4/8/2014,13:45:00,4/8/2014,13:50:00,4/8/2014,DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, SALE 5",ATTEMPTED,FELONY,BRONX,52,INSIDE,STREET,40.8642,-73.9012
3,5/17/2014,21:17:00,5/17/2014,21:24:00,5/17/2014,FELONY ASSAULT,"ASSAULT 2,1,PEACE OFFICER",ATTEMPTED,FELONY,BRONX,40,INSIDE,STREET,40.8131,-73.9082
4,4/26/2014,00:00:00,4/26/2014,00:01:00,4/26/2014,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",ATTEMPTED,FELONY,BRONX,41,INSIDE,STREET,40.8146,-73.8859


In [5]:
#Filter the data to fetch only crimes with status 'Completed'
from data_utility import filterData
completed_crimes = filterData(crimes_original,7,'COMPLETED')

In [6]:
#Drop unwanted comlumns
dropped_columns = completed_crimes.drop([0,1,2,3,4,6,11,13,14,5,7,10],axis=1)

#Rename the columns
dropped_columns.columns = ['TYPE OF CRIME','BOROUGHS','PREMISES']
dropped_columns.head()

,TYPE OF CRIME,BOROUGHS,PREMISES
2164,FELONY,BRONX,BAR/NIGHT CLUB
2165,FELONY,QUEENS,nan
2166,FELONY,MANHATTAN,OTHER
2167,MISDEMEANOR,QUEENS,RESIDENCE-HOUSE
2168,MISDEMEANOR,MANHATTAN,OTHER


In [7]:
#Since population count is carried once in a decade, get the mean population of the year 2010 and 2020 for each borough
bronxPop = getMeanPopulation('Bronx', 1)
brookylnPop = getMeanPopulation('Brooklyn', 2)
manhattanPop = getMeanPopulation('Manhattan', 3)
queensPop = getMeanPopulation('Queens', 4)
statIslandPop = getMeanPopulation('Staten Island', 5)
total_population = bronxPop+brookylnPop+manhattanPop+queensPop+statIslandPop


In [8]:
#Set Type of crime to a variable
felony_val = 'FELONY'
mis_val = 'MISDEMEANOR'
vio_val = 'VIOLATION'

#Fetch crimes that are categorized under Felony
data = filterData(dropped_columns,'TYPE OF CRIME',felony_val)
data.head()

,TYPE OF CRIME,BOROUGHS,PREMISES
2164,FELONY,BRONX,BAR/NIGHT CLUB
2165,FELONY,QUEENS,nan
2166,FELONY,MANHATTAN,OTHER
2171,FELONY,BRONX,STREET
2176,FELONY,MANHATTAN,STREET


In [9]:
#Function to highlight the max value in a row
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [10]:
#Function that returns data in the form of Pivot table for the corresponding type of crime
def createPivotTable(data,levelName):
    
    #set level name for pivot table
    data.columns = [levelName,'BOROUGHS','PREMISES 
                    ']
    
    #Convert the Felony dataset into a pivot table as per Boroughs and Premises
    fel_pivot = data.pivot_table(values = [levelName], index = ['PREMISES'], columns = ['BOROUGHS'], aggfunc=np.size,fill_value=0,margins=True)
    
    #Convert the values from int64 to float64
    for i in range(6):
        fel_pivot[fel_pivot.columns[0][0], fel_pivot.columns[i][1]] = fel_pivot[fel_pivot.columns[0][0],fel_pivot.columns[i][1]].astype('float64')
        
    #Divide the number of crimes with the mean population in each premises for the corresponding Borough and round it upto 6 decimal digits
    for idx, row in fel_pivot.iterrows():
                  row[0]=round(row[0]/bronxPop,6)
                  row[1]=round(row[1]/brookylnPop,6)
                  row[2]=round(row[2]/manhattanPop,6)
                  row[3]=round(row[3]/queensPop,6)
                  row[4]=round(row[4]/statIslandPop,6)
                  row[5]=round(row[5]/total_population,6)
                    
    #Sort the pivot table by All(Total Crime for each premise and borough)
    sort_pivot = fel_pivot.reindex(fel_pivot[levelName].sort_values(by='All', ascending=False).index)
    
    #Highlight max values for each of the Boroughs
    temp = sort_pivot.drop(['All'])
    sort_pivot_final = temp.style.apply(highlight_max)
    
    return sort_pivot_final
    

In [11]:
#Create a pivot table for Felonious Crimes under Boroughs and Premises
createPivotTable(data,felony_val)

In [12]:
#Fetch crimes that are categorized under Misdemeanor
data_mis = filterData(dropped_columns,'TYPE OF CRIME',mis_val)
createPivotTable(data_mis,mis_val)

In [15]:
#Fetch crimes that are categorized under Violation
data_vio = filterData(dropped_columns,'TYPE OF CRIME',vio_val)
temp =createPivotTable(data_vio,vio_val)

In [26]:
temp.columns[5][1]

'All'

In [34]:
temp.index

Index(['RESIDENCE - APT. HOUSE', 'STREET', 'RESIDENCE-HOUSE',
       'RESIDENCE - PUBLIC HOUSING', 'PUBLIC SCHOOL', 'OTHER',
       'COMMERCIAL BUILDING', 'TRANSIT - NYC SUBWAY', 'GROCERY/BODEGA',
       'RESTAURANT/DINER', 'HOSPITAL', 'PUBLIC BUILDING', 'CHAIN STORE',
       'BAR/NIGHT CLUB', 'PARK/PLAYGROUND', 'FAST FOOD',
       'DOCTOR/DENTIST OFFICE', 'nan', 'BUS (NYC TRANSIT)', 'HOTEL/MOTEL',
       'BEAUTY & NAIL SALON', 'STORE UNCLASSIFIED', 'DEPARTMENT STORE',
       'SMALL MERCHANT', 'PARKING LOT/GARAGE (PUBLIC)', 'FOOD SUPERMARKET',
       'CLOTHING/BOUTIQUE', 'DRUG STORE', 'CHURCH',
       'PARKING LOT/GARAGE (PRIVATE)', 'DRY CLEANER/LAUNDRY', 'BANK',
       'GYM/FITNESS FACILITY', 'PRIVATE/PAROCHIAL SCHOOL', 'GAS STATION',
       'CANDY STORE', 'FACTORY/WAREHOUSE', 'BUS (OTHER)', 'BUS STOP',
       'TELECOMM. STORE', 'OPEN AREAS (OPEN LOTS)', 'VARIETY STORE',
       'CONSTRUCTION SITE', 'TAXI (YELLOW LICENSED)', 'SOCIAL CLUB/POLICY',
       'FERRY/FERRY TERMINAL', 'CHECK C

In [37]:
temp.index.name

'PREMISES'

In [45]:
temp.columns.names[1]


'BOROUGHS'